___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# 얼굴 감지(Face Detection)

[MediaPipe](https://google.github.io/mediapipe/)의 [Face Mesh](https://google.github.io/mediapipe/solutions/face_mesh) 사용  

활용 사례: [Flappybird](https://www.youtube.com/shorts/3ywKjfO24J0), [SparkAR Gallery](https://www.facebook.com/sparkarhub/gallery?ref=spark_website), [운전자 졸음 감지](https://www.youtube.com/watch?v=Q23K7G1gJgY) 등


[Landmark index map](https://github.com/google/mediapipe/blob/a908d668c730da128dfa8d9f6bd25d519d006692/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png)

In [ ]:
import cv2
import mediapipe as mp

def find_landmarks(img_bgr, face_mesh):
    """Normalized image space to pixel space"""

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)
    landmarks = []
    if results.multi_face_landmarks:
        for person in results.multi_face_landmarks:
            face = []
            for id, lm in enumerate(person.landmark):
                ih, iw, _ = img_bgr.shape
                x, y = int(lm.x * iw), int(lm.y * ih)
                face.append([x, y])
            landmarks.append(face)
    return landmarks

cap = cv2.VideoCapture("jmhong_face.mp4")
#cap = cv2.VideoCapture(0)

with mp.solutions.face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as face_mesh:

    while cap.isOpened():

        success, image = cap.read()
        if not success:            
            break

        landmarks = find_landmarks(image, face_mesh)

        if not landmarks:
            print("No person detected")
            continue

        # 모든 랜드마크 그리기
        # for xy in landmarks[0]: # [0] is for the first person
        #     cv2.circle(image, xy, 2, (255, 255, 255), cv2.FILLED)

        # 특정 랜드마크만 그리기
        cv2.circle(image, landmarks[0][159], 2, (255, 255, 255), cv2.FILLED)
        cv2.circle(image, landmarks[0][145], 2, (255, 255, 255), cv2.FILLED)

        cv2.imshow("MediaPipe Face Mesh", image)
        if cv2.waitKey(5) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### [실습] 윙크 감지

힌트: 두 점 사이의 거리(Euclidean distance) 구하는 방법은 수학 시간에 배운 그대로 적용하면 됩니다. 검색해도 많이 나옵니다.